In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Практикум 3 семестр/data_sept_analysis(more_columns).csv')

In [4]:
df['Дата и время события'] = pd.to_datetime(df['Дата и время события'])

In [30]:
df_sorted_desc = df.sort_values(by='Длина сессии в минутах', ascending=False)
df_sorted_desc.head(10)

,Unnamed: 0,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,ОС,age_back,gender,row_id,Ушел из приложения,is_churn,is_finish,age_group,Длина сессии в минутах
1378997,1378997,2025-10-22 08:59:27+03:00,Еще,Открытие экрана,Переход к экрану,35622,10000000016,Apple,iPhone 15 Pro,phone,iOS,53.0,М,1590508,0,0,1,51–60,71993.150000
863911,863911,2025-09-02 09:06:18+03:00,Еще,Открытие экрана,Переход к экрану,35622,10000000016,Apple,iPhone 15 Pro,phone,iOS,53.0,М,1000707,0,0,0,51–60,71993.150000
1000334,1000334,2025-09-02 19:17:32+03:00,Еще,Открытие экрана,Переход к экрану,47939,10000000047,Apple,iPhone 12 Mini,phone,iOS,28.0,Ж,1157511,0,0,0,25–30,70674.300000
660406,660406,2025-10-21 21:11:50+03:00,Еще,Открытие экрана,Переход к экрану,47939,10000000047,Apple,iPhone 12 Mini,phone,iOS,28.0,Ж,761092,0,0,1,25–30,70674.300000
660407,660407,2025-10-21 12:20:29+03:00,Еще,Открытие экрана,Переход к экрану,47939,10000000045,Apple,iPhone 12 Mini,phone,iOS,28.0,Ж,761093,0,0,1,25–30,70649.600000
1000333,1000333,2025-09-02 10:50:53+03:00,Еще,Открытие экрана,Переход к экрану,47939,10000000045,Apple,iPhone 12 Mini,phone,iOS,28.0,Ж,1157510,0,0,0,25–30,70649.600000
267405,267405,2025-09-01 11:38:42+03:00,Еще,Открытие экрана,Переход к экрану,42189,10000000010,Apple,iPhone 16 Pro Max,phone,iOS,30.0,Ж,306645,0,0,0,25–30,70574.433333
2683141,2683141,2025-10-20 11:53:08+03:00,Еще,Открытие экрана,Переход к экрану,42189,10000000010,Apple,iPhone 16 Pro Max,phone,iOS,30.0,Ж,3163230,0,0,1,25–30,70574.433333
2683138,2683138,2025-10-20 16:50:16+03:00,Еще,Открытие экрана,Переход к экрану,42189,10000000011,Apple,iPhone 16 Pro Max,phone,iOS,30.0,Ж,3163227,0,0,1,25–30,70455.750000
267404,267404,2025-09-01 18:34:31+03:00,Еще,Открытие экрана,Переход к экрану,42189,10000000011,Apple,iPhone 16 Pro Max,phone,iOS,30.0,Ж,306644,0,0,0,25–30,70455.750000


In [5]:
#исключаем аномальные (те что от 60 минут)
df_anomalies = df[df['Длина сессии в минутах'] >= 60]
df_clean = df[df['Длина сессии в минутах'] < 60]

In [6]:
df_clean['age_group'] = df_clean['age_group'].fillna('unknown')

/tmp/ipython-input-1356404354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['age_group'] = df_clean['age_group'].fillna('unknown')


In [7]:
df_anomalies.shape[0]

4392

In [7]:
events_per_session = (
    df_clean
    .groupby(["Идентификатор устройства", "Номер сессии в рамках устройства"])
    .size()
    .reset_index(name="events_cnt")
)

In [8]:
n_sessions = events_per_session.shape[0]
few_events = events_per_session[events_per_session['events_cnt']<=2].shape[0]
lot_events = events_per_session[events_per_session['events_cnt']>2].shape[0]
print(f'Из общего количества сессий {n_sessions}, {round(few_events/n_sessions*100, 2)}% состоят из 1-2 действий и {round(lot_events/n_sessions*100, 2)}% - из более чем 2.')

Из общего количества сессий 890205, 65.63% состоят из 1-2 действий и 34.37% - из более чем 2.


In [9]:
df_clean = df_clean.merge(
    events_per_session,
    on=["Идентификатор устройства", "Номер сессии в рамках устройства"],
    how="left"
)

In [10]:
assert df_clean["events_cnt"].isna().sum() == 0

In [11]:
df_clean['short_session'] = df_clean['events_cnt'] <= 2

result = (
    df_clean
    .groupby(['age_group', 'is_churn'])['short_session']
    .mean()
    .reset_index(name='share_short_sessions')
)

result['share_short_sessions'] *= 100

In [12]:
result

,age_group,is_churn,share_short_sessions
0,18–24,0,37.686577
1,18–24,1,74.345364
2,25–30,0,34.914348
3,25–30,1,74.236111
4,31-40,0,27.456713
5,31-40,1,67.852129
6,41–50,0,24.053949
7,41–50,1,64.701390
8,51–60,0,23.556432
9,51–60,1,63.561563


In [13]:
len(df_clean.groupby(["Идентификатор устройства", "Номер сессии в рамках устройства"]).tail(1))

890205

In [14]:
short_df = df_clean[df_clean['events_cnt'] <= 2]

counts = (
    short_df
    .groupby(['age_group', 'is_churn'])
    .size()
    .reset_index(name='n_short_sessions')
)

total_sessions = len(df_clean.groupby(["Идентификатор устройства", "Номер сессии в рамках устройства"]).tail(1))

counts['share_from_all_short_sessions'] = (
    counts['n_short_sessions'] / total_sessions * 100
)

counts

,age_group,is_churn,n_short_sessions,share_from_all_short_sessions
0,18–24,0,27319,3.068844
1,18–24,1,2101,0.236013
2,25–30,0,70520,7.921771
3,25–30,1,4276,0.480339
4,31-40,0,234879,26.384821
5,31-40,1,14500,1.628838
6,41–50,0,200246,22.494369
7,41–50,1,12708,1.427536
8,51–60,0,110874,12.454884
9,51–60,1,6489,0.728933


In [16]:
leave_ios = df_clean[(df_clean['Ушел из приложения']==1) & (df_clean['ОС']=='iOS')].shape[0]
leave_android = df_clean[(df_clean['Ушел из приложения']==1) & (df_clean['ОС']=='Android')].shape[0]
stay_ios = df_clean[(df_clean['Ушел из приложения']==0) & (df_clean['ОС']=='iOS')].shape[0]
stay_android = df_clean[(df_clean['Ушел из приложения']==0) & (df_clean['ОС']=='Android')].shape[0]
print(stay_ios/leave_ios, stay_android/leave_android)

4.875116963426045 4.4285326967225815


In [33]:
df_sorted_desc = df_clean.sort_values(by='Длина сессии в минутах', ascending=True)
df_sorted_desc.head(10)

,Unnamed: 0,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,...,gender,row_id,Ушел из приложения,is_churn,is_finish,age_group,Длина сессии в минутах,events_cnt,short_session,mean_time_per_event
2841207,2845599,2025-10-09 15:54:54+03:00,Еще,Открытие экрана,Переход к экрану,103919,10000000028,Apple,iPhone 16 Pro Max,phone,...,М,3360266,0,0,1,41–50,0.0,1,True,0.0
2841239,2845631,2025-10-09 01:24:09+03:00,Услуги,Открытие экрана,Открытие экрана,86064,10000000054,Xiaomi,Xiaomi 13 Lite,phone,...,М,3360345,0,0,1,41–50,0.0,1,True,0.0
2841250,2845642,2025-10-09 13:43:37+03:00,Еще,Открытие экрана,Переход к экрану,90212,10000000029,Apple,iPhone 16 Pro,phone,...,Ж,3360356,0,0,1,25–30,0.0,1,True,0.0
2841251,2845643,2025-10-09 21:46:54+03:00,Еще,Открытие экрана,Переход к экрану,86067,10000000037,Apple,iPhone 16 Pro Max,phone,...,М,3360358,0,0,1,18–24,0.0,1,True,0.0
2841252,2845644,2025-10-09 14:18:18+03:00,Еще,Открытие экрана,Переход к экрану,114201,10000000063,Apple,iPhone 12 Pro Max,phone,...,Ж,3360360,0,0,1,31-40,0.0,1,True,0.0
2841288,2845680,2025-10-09 16:09:09+03:00,Еще,Открытие экрана,Переход к экрану,61597,10000000018,Apple,iPhone 16 Pro Max,phone,...,Ж,3360409,0,0,1,41–50,0.0,1,True,0.0
113,113,2025-09-29 06:48:58+03:00,Еще,Открытие экрана,Переход к экрану,362,10000000076,Huawei,AQM-LX1,phone,...,М,113,1,1,1,41–50,0.0,1,True,0.0
114,114,2025-09-29 12:13:59+03:00,Еще,Открытие экрана,Переход к экрану,363,10000000016,Apple,iPhone 13 Pro,phone,...,Ж,114,0,0,1,41–50,0.0,1,True,0.0
1028607,1030401,2025-09-19 20:28:07+03:00,Еще,Открытие экрана,Переход к экрану,24514,10000000349,Apple,iPhone 14 Plus,phone,...,Ж,1192012,0,0,1,31-40,0.0,1,True,0.0
2841432,2845824,2025-10-09 18:56:36+03:00,Еще,Открытие экрана,Переход к экрану,152365,10000000054,Apple,iPhone XR,phone,...,Ж,3360646,0,0,1,41–50,0.0,1,True,0.0


In [15]:
df_clean['mean_time_per_event'] = round(df_clean['Длина сессии в минутах'] / df_clean['events_cnt'], 1)

In [74]:
import numpy as np
bins = [0, 2, 5, 10, 30, np.inf]
labels = ['1–2', '3–5', '6–10', '11-30', "31+"]

df_clean['session_length_bin'] = pd.cut(
    df_clean['events_cnt'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

result = (
    df_clean
    .groupby(
        ['Ушел из приложения', 'session_length_bin'],
        observed=True
    )
    .size()
    .reset_index(name='n_sessions')
)

result['share_sessions'] = (
    result
    .groupby(['Ушел из приложения'])['n_sessions']
    .transform(lambda x: x / x.sum() * 100)
)

display(result)

,Ушел из приложения,session_length_bin,n_sessions,share_sessions
0,0,1–2,624734,26.741357
1,0,3–5,541395,23.174082
2,0,6–10,509981,21.829425
3,0,11-30,554407,23.731053
4,0,31+,105692,4.524082
5,1,1–2,138018,27.318017
6,1,3–5,125249,24.790639
7,1,6–10,108854,21.545563
8,1,11-30,113116,22.389144
9,1,31+,19990,3.956637


In [75]:
pivot = result.pivot_table(
    index=['Ушел из приложения'],
    columns='session_length_bin',
    values='share_sessions'
)

pivot


/tmp/ipython-input-4035072674.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = result.pivot_table(


session_length_bin,1–2,3–5,6–10,11-30,31+
Ушел из приложения,,,,,
0,26.741357,23.174082,21.829425,23.731053,4.524082
1,27.318017,24.790639,21.545563,22.389144,3.956637


In [18]:
import numpy as np
bins = [0, 0.05, 0.1, 0.2, 0.5, 1, 2, 4, np.inf]
labels = ['0-0.05', '0.05-0.1', '0.1–0.2', '0.2-0.5', '0.5-1', '1-2', '2-4', "4+"]

df_clean['session_length_bin'] = pd.cut(
    df_clean['mean_time_per_event'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

counts = (
    df_clean
    .groupby(['session_length_bin', 'Ушел из приложения'])
    .size()
    .reset_index(name='n_events')
)

total_events = len(df_clean)

counts['share_from_all_events'] = (
    counts['n_events'] / total_events * 100
)

pivot = counts.pivot_table(
    index='session_length_bin',
    columns='Ушел из приложения',
    values='share_from_all_events'
)

pivot

/tmp/ipython-input-285277996.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['session_length_bin', 'Ушел из приложения'])
/tmp/ipython-input-285277996.py:26: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = counts.pivot_table(


Ушел из приложения,0,1
session_length_bin,,
0-0.05,24.757447,4.255067
0.05-0.1,25.635101,4.475800
0.1–0.2,14.661777,3.509176
0.2-0.5,12.665532,3.821061
0.5-1,3.417673,1.281183
1-2,0.912496,0.365625
2-4,0.145279,0.062574
4+,0.024002,0.010206


In [16]:
df_clean['number_of_sessions'] = (
    df_clean
    .groupby("Идентификатор устройства")["Номер сессии в рамках устройства"]
    .transform('nunique')
)

In [17]:
#количество сессий на пользователя в зависимости от группы ушел/остался
users = (
    df_clean
    .drop_duplicates(subset="Идентификатор устройства")
    [['Идентификатор устройства', 'number_of_sessions', 'Ушел из приложения']]
)
stats = (
    users
    .groupby('Ушел из приложения')['number_of_sessions']
    .agg(['mean', 'median', 'count'])
    .reset_index()
)

stats

,Ушел из приложения,mean,median,count
0,0,10.631810,6.0,68557
1,1,2.397634,2.0,67283


In [18]:
df_sessions = (
    df_clean
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "Длина сессии в минутах",
        "Ушел из приложения"
    ]]
)
user_session_stats = (
    df_sessions
    .groupby("Идентификатор устройства")
    .agg(
        number_of_sessions=("Номер сессии в рамках устройства", "count"),
        total_time_minutes=("Длина сессии в минутах", "sum"),
        avg_session_length_minutes=("Длина сессии в минутах", "mean"),
        user_churn=("Ушел из приложения", "max")
    )
    .reset_index()
)

user_session_stats


,Идентификатор устройства,number_of_sessions,total_time_minutes,avg_session_length_minutes,user_churn
0,1,70,62.583333,0.894048,0
1,2,44,9.966667,0.226515,0
2,6,7,3.150000,0.450000,0
3,8,86,89.200000,1.037209,0
4,9,12,13.716667,1.143056,0
...,...,...,...,...,...
135835,180284,1,0.166667,0.166667,1
135836,180285,1,0.100000,0.100000,1
135837,180286,2,2.833333,1.416667,1
135838,180287,1,3.216667,3.216667,1


In [19]:
#количество сессий, общее время в приложении и длина сессии в минутах для разных групп
user_session_stats.groupby("user_churn")[[
    "number_of_sessions",
    "total_time_minutes",
    "avg_session_length_minutes"
]].agg(["mean", "median", "count"])


number_of_sessions               total_time_minutes            \
                         mean median  count               mean    median   
user_churn                                                                 
0                   10.631810    6.0  68557           6.004711  2.866667   
1                    2.397634    2.0  67283           1.791017  0.533333   

                  avg_session_length_minutes                   
            count                       mean    median  count  
user_churn                                                     
0           68557                   0.676703  0.425758  68557  
1           67283                   0.808651  0.266667  67283

In [20]:
#процент пользователей только с ультракороткими сессиями <= 6 секунд
SHORT_SESSION_THRESHOLD = 0.1

users = (
    df_sessions
    .groupby("Идентификатор устройства")
    .agg(
        avg_session_length_minutes=("Длина сессии в минутах", "mean"),
        user_churn=("Ушел из приложения", "max")
    )
    .reset_index()
)
users['has_short_sessions'] = (
    users['avg_session_length_minutes'] <= SHORT_SESSION_THRESHOLD
)
result = (
    users
    .groupby('user_churn')['has_short_sessions']
    .mean()
    .reset_index(name='share_users_with_short_sessions')
)

result['share_users_with_short_sessions'] *= 100
result


,user_churn,share_users_with_short_sessions
0,0,22.915238
1,1,40.222642


In [32]:
#сессии из 1-2 действий из общего числа сессий для каждой группы
df_sessions = (
    df_clean
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "events_cnt",
        "Ушел из приложения"
    ]]
)
df_sessions['short_session'] = df_sessions['events_cnt'] <= 2
result = (
    df_sessions
    .groupby('Ушел из приложения')['short_session']
    .mean()
    .reset_index(name='share_short_sessions')
)

result['share_short_sessions'] *= 100
result


,Ушел из приложения,share_short_sessions
0,0,65.696783
1,1,65.303744


In [21]:
#процент пользователей только с сессиями из 1-2 действий
df_sessions = (
    df_clean
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "events_cnt",
        "Ушел из приложения"
    ]]
)
user_only_short = (
    df_sessions
    .groupby("Идентификатор устройства")
    .agg(
        max_events_in_session=('events_cnt', 'max'),
        user_churn=('Ушел из приложения', 'max')
    )
    .reset_index()
)
user_only_short['only_short_sessions'] = (
    user_only_short['max_events_in_session'] <= 2
)
counts = (
    user_only_short
    .groupby(['user_churn', 'only_short_sessions'])
    .size()
    .reset_index(name='n_users')
)

shares = (
    user_only_short
    .groupby('user_churn')['only_short_sessions']
    .mean()
    .reset_index(name='share_users_only_short_sessions')
)

shares['share_users_only_short_sessions'] *= 100
shares

,user_churn,share_users_only_short_sessions
0,0,26.548711
1,1,50.388657


In [22]:
#пользователи с сессиями больше 1-2 действий
users_with_long_sessions = user_only_short.loc[
    ~user_only_short['only_short_sessions'],
    'Идентификатор устройства'
]
df_clean_long_users = df_clean[
    df_clean['Идентификатор устройства'].isin(users_with_long_sessions)
].copy()

In [23]:
df_sessions = (
    df_clean_long_users
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "Длина сессии в минутах",
        "Ушел из приложения"
    ]]
)
user_session_stats = (
    df_sessions
    .groupby("Идентификатор устройства")
    .agg(
        number_of_sessions=("Номер сессии в рамках устройства", "count"),
        total_time_minutes=("Длина сессии в минутах", "sum"),
        avg_session_length_minutes=("Длина сессии в минутах", "mean"),
        user_churn=("Ушел из приложения", "max")
    )
    .reset_index()
)

user_session_stats.groupby("user_churn")[[
    "number_of_sessions",
    "total_time_minutes",
    "avg_session_length_minutes"
]].agg(["mean", "median", "count"])

number_of_sessions               total_time_minutes            \
                         mean median  count               mean    median   
user_churn                                                                 
0                   11.681150    7.0  50356           7.971305  4.683333   
1                    2.952876    2.0  33380           3.336400  2.083333   

                  avg_session_length_minutes                   
            count                       mean    median  count  
user_churn                                                     
0           50356                    0.88016  0.633333  50356  
1           33380                    1.46469  0.908796  33380

In [26]:
#процент пользователей с разным количеством сессий для ушедших
churn_users = df_clean_long_users[df_clean_long_users['Ушел из приложения'] == 1].copy()

import numpy as np
import pandas as pd

bins = [0, 1, 2, 3, 4, np.inf]
labels = ['1', '2', '3', '4', '5+']

churn_users['sessions_bin'] = pd.cut(
    churn_users['number_of_sessions'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)
dist = (
    churn_users['sessions_bin']
    .value_counts(normalize=True)
    .sort_index()
    * 100
)

dist_df = dist.reset_index()
dist_df.columns = ['number_of_sessions', 'share_churn_users']
dist_df

,number_of_sessions,share_churn_users
0,1,16.483179
1,2,18.849301
2,3,15.565976
3,4,11.685320
4,5+,37.416224


In [25]:
#процент пользователей с разным количеством сессий для оставшихся
churn_users = df_clean_long_users[df_clean_long_users['Ушел из приложения'] == 0].copy()

import numpy as np
import pandas as pd

bins = [0, 1, 2, 3, 4, np.inf]
labels = ['1', '2', '3', '4', '5+']

churn_users['sessions_bin'] = pd.cut(
    churn_users['number_of_sessions'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)
dist = (
    churn_users['sessions_bin']
    .value_counts(normalize=True)
    .sort_index()
    * 100
)

dist_df = dist.reset_index()
dist_df.columns = ['number_of_sessions', 'share_churn_users']
dist_df

,number_of_sessions,share_churn_users
0,1,0.000781
1,2,1.533706
2,3,3.142753
3,4,3.982719
4,5+,91.340041


Если пользователь дошёл до 5-й сессии, вероятность того, что он останется, резко возрастает.
Анализ числа сессий показывает, что достижение пятой сессии является критическим этапом жизненного цикла пользователя. Более 90% пользователей, продолжающих использование приложения, имеют не менее пяти сессий, тогда как среди ушедших пользователей доля таких пользователей составляет лишь около 37%. Это указывает на наличие порога активации, преодоление которого существенно снижает вероятность ухода.

In [51]:
df_clean_long_users

,Unnamed: 0,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,...,Ушел из приложения,is_churn,is_finish,age_group,Длина сессии в минутах,events_cnt,short_session,mean_time_per_event,session_length_bin,number_of_sessions
0,0,2025-09-29 10:20:27+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,...,0,0,0,61–70,3.533333,9,False,0.4,0.2-0.5,12
1,1,2025-09-29 10:21:56+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,...,0,0,0,61–70,3.533333,9,False,0.4,0.2-0.5,12
2,2,2025-09-29 10:22:01+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',339,10000000009,Redmi,Redmi Note 12,phone,...,0,0,0,61–70,3.533333,9,False,0.4,0.2-0.5,12
3,3,2025-09-29 10:23:49+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,...,0,0,1,61–70,3.533333,9,False,0.4,0.2-0.5,12
4,4,2025-09-29 10:20:17+03:00,Новая заявка,Выбор квартиры,Тап на квартиру,339,10000000009,Redmi,Redmi Note 12,phone,...,0,0,0,61–70,3.533333,9,False,0.4,0.2-0.5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2841431,2845823,2025-10-09 16:18:31+03:00,Новое ОСС,Открытие экрана,Переход к экрану,123401,10000000010,Apple,iPhone 7 Plus,phone,...,0,0,1,41–50,3.733333,6,False,0.6,0.5-1,7
2841432,2845824,2025-10-09 18:56:36+03:00,Еще,Открытие экрана,Переход к экрану,152365,10000000054,Apple,iPhone XR,phone,...,0,0,1,41–50,0.000000,1,True,0.0,0-0.05,3
2841433,2845825,2025-10-09 14:41:10+03:00,Еще,Открытие экрана,Переход к экрану,99472,10000000098,Apple,iPhone 14 Pro Max,phone,...,0,0,0,31-40,0.100000,3,False,0.0,0-0.05,10
2841434,2845826,2025-10-09 14:41:16+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',99472,10000000098,Apple,iPhone 14 Pro Max,phone,...,0,0,0,31-40,0.100000,3,False,0.0,0-0.05,10


In [27]:
df_sessions = (
    df_clean_long_users
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "events_cnt",
        "Длина сессии в минутах",
        "Ушел из приложения"
    ]]
    .copy()
)
df_sessions = df_sessions.sort_values(
    ["Идентификатор устройства", "Номер сессии в рамках устройства"]
)

df_sessions["session_rank"] = (
    df_sessions
    .groupby("Идентификатор устройства")
    .cumcount() + 1
)
session_progression = (
    df_sessions[df_sessions["session_rank"] <= 5]
    .groupby(["Ушел из приложения", "session_rank"])
    .agg(
        mean_events=("events_cnt", "mean"),
        median_events=("events_cnt", "median"),
        mean_session_length=("Длина сессии в минутах", "mean"),
        median_session_length=("Длина сессии в минутах", "median"),
        n_sessions=("session_rank", "count")
    )
    .reset_index()
)

session_progression

,Ушел из приложения,session_rank,mean_events,median_events,mean_session_length,median_session_length,n_sessions
0,0,1,4.065414,2.0,1.161106,0.283333,50356
1,0,2,3.778067,2.0,0.914206,0.183333,50353
2,0,3,3.679703,2.0,0.814487,0.150000,46741
3,0,4,3.687434,2.0,0.779249,0.133333,41380
4,0,5,3.679646,2.0,0.739127,0.116667,35979
5,1,1,5.228250,4.0,1.661371,0.866667,33380
6,1,2,4.201955,3.0,1.053325,0.333333,22203
7,1,3,3.996281,2.0,0.906384,0.216667,13713
8,1,4,3.781693,2.0,0.805311,0.183333,8598
9,1,5,3.726035,2.0,0.776816,0.150000,5508


Анализ динамики пользовательских сессий показал, что пользователи, покинувшие приложение, в первой сессии демонстрируют более высокую активность и проводят больше времени, чем пользователи, продолжающие использование. Однако уже ко второй сессии у них наблюдается резкое снижение длительности и насыщенности взаимодействия, что может свидетельствовать о фрустрации и отсутствии устойчивого сценария использования. Пользователи, продолжающие использование приложения, демонстрируют более компактное и стабильное поведение с постепенным снижением длительности сессий, что указывает на освоение интерфейса.

In [48]:
import pandas as pd
import numpy as np

bins = [0, 0.05, 0.1, 0.2, 0.5, np.inf]  # в минутах
labels = [
    '0–3 сек',
    '3–6 сек',
    '6–12 сек',
    '12–30 сек',
    '30+ секунд'
]

df_clean_long_users['event_time_bin'] = pd.cut(
    df_clean_long_users['mean_time_per_event'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

In [49]:
age_event_dist = (
    df_clean_long_users
    .groupby(['age_group', 'event_time_bin'], observed=True)
    .size()
    .reset_index(name='n_events')
)

age_event_dist['share_events'] = (
    age_event_dist
    .groupby('age_group')['n_events']
    .transform(lambda x: x / x.sum() * 100)
)

age_event_dist


,age_group,event_time_bin,n_events,share_events
0,18–24,0–3 сек,19723,31.313308
1,18–24,3–6 сек,20496,32.540565
2,18–24,6–12 сек,10997,17.459435
3,18–24,12–30 сек,8898,14.126949
4,18–24,30+ секунд,2872,4.559743
5,25–30,0–3 сек,53113,29.199813
6,25–30,3–6 сек,62938,34.601281
7,25–30,6–12 сек,33726,18.541466
8,25–30,12–30 сек,24647,13.550125
9,25–30,30+ секунд,7471,4.107315


In [66]:
users_long = (
    df_clean_long_users
    .drop_duplicates(subset=["Идентификатор устройства"])
    [["Идентификатор устройства", "age_group", "Ушел из приложения"]]
)

age_churn_rate = (
    users_long
    .groupby("age_group", observed=True)["Ушел из приложения"]
    .mean()
    .mul(100)
    .reset_index(name="churn_rate_percent")
)

age_churn_rate

,age_group,churn_rate_percent
0,18–24,47.785453
1,25–30,39.359699
2,31-40,39.902676
3,41–50,40.028615
4,51–60,37.775378
5,61–70,38.589861
6,71–101,43.276284
7,unknown,82.916667


In [62]:
threshold = (
    df_clean_long_users
    .loc[df_clean_long_users['Ушел из приложения'] == 1, 'mean_time_per_event']
    .quantile(0.75)
)

threshold

np.float64(0.3)

In [63]:
long_event_churn = df_clean_long_users[
    (df_clean_long_users['Ушел из приложения'] == 1) &
    (df_clean_long_users['mean_time_per_event'] >= threshold)
]
top_devices = (
    long_event_churn['ОС']
    .value_counts()
    .head(15)
    .reset_index()
)

top_devices.columns = ['ОС', 'n_events']
top_devices

,ОС,n_events
0,Android,82892
1,iOS,64752


Технические особенности устройства влияют на длительность взаимодействия, но не являются драйвером churn (пользователи IOS в среднем тратят больше времени на одно действие).

In [28]:
# определяем номер первой сессии для каждого пользователя
first_sessions = (
    df_clean_long_users
    .groupby("Идентификатор устройства")["Номер сессии в рамках устройства"]
    .min()
    .reset_index(name="first_session_id")
)

# оставляем только события первой сессии
df_first_session = (
    df_clean_long_users
    .merge(first_sessions, on="Идентификатор устройства")
    .query("`Номер сессии в рамках устройства` == first_session_id")
    .copy()
)

In [30]:
top_screens = (
    df_first_session
    .groupby(["Ушел из приложения", "Экран"])
    .size()
    .reset_index(name="events_cnt")
    .sort_values(["Ушел из приложения", "events_cnt"], ascending=[True, False])
)

top_screens.groupby("Ушел из приложения").head(10)

,Ушел из приложения,Экран,events_cnt
2,0,Еще,100543
6,0,Новая заявка,66294
0,0,Важное,25257
10,0,Услуги,4711
4,0,Мои баллы,3794
8,0,Объявления,1670
7,0,Новое ОСС,1582
9,0,Связаться с владельцем ТС,799
3,0,Заявки,48
1,0,Гостевой доступ,19


In [32]:
last_events_first_session = (
    df_first_session
    .sort_values("Дата и время события")
    .groupby(["Идентификатор устройства"])
    .tail(1)
)
top_last_screens = (
    last_events_first_session
    .groupby(["Ушел из приложения", "Экран"])
    .size()
    .reset_index(name="n_users")
    .sort_values(["Ушел из приложения", "n_users"], ascending=[True, False])
)

top_last_screens.groupby("Ушел из приложения").head(10)

,Ушел из приложения,Экран,n_users
2,0,Еще,32772
0,0,Важное,7716
6,0,Новая заявка,7287
10,0,Услуги,861
4,0,Мои баллы,793
7,0,Новое ОСС,403
9,0,Связаться с владельцем ТС,387
8,0,Объявления,128
3,0,Заявки,6
1,0,Гостевой доступ,2


In [33]:
top_features = (
    df_first_session
    .groupby(["Ушел из приложения", "Функционал"])
    .size()
    .reset_index(name="events_cnt")
    .sort_values(["Ушел из приложения", "events_cnt"], ascending=[True, False])
)

top_features.groupby("Ушел из приложения").head(10)


,Ушел из приложения,Функционал,events_cnt
23,0,Открытие экрана,82085
69,0,Просмотр уведомления,25257
15,0,Выбор через теги,14094
1,0,Возврат на предыдущий этап,13390
39,0,Переход в раздел 'Опросы и собрания собственни...,8241
32,0,Переход в раздел 'Заявки',7450
40,0,Переход в раздел 'Приборы учета',6120
16,0,Вызов поиска,5794
28,0,Отправка заявки,5541
35,0,Переход в раздел 'Мои платежи',4446


In [34]:
top_last_features = (
    last_events_first_session
    .groupby(["Ушел из приложения", "Функционал"])
    .size()
    .reset_index(name="n_users")
    .sort_values(["Ушел из приложения", "n_users"], ascending=[True, False])
)

top_last_features.groupby("Ушел из приложения").head(10)

,Ушел из приложения,Функционал,n_users
22,0,Открытие экрана,22433
62,0,Просмотр уведомления,7716
36,0,Переход в раздел 'Опросы и собрания собственни...,3705
37,0,Переход в раздел 'Приборы учета',3276
26,0,Отправка заявки,2386
32,0,Переход в раздел 'Мои платежи',2301
29,0,Переход в раздел 'Заявки',1671
1,0,Возврат на предыдущий этап,1262
15,0,Выбор через теги,727
38,0,Переход в раздел 'Профиль',683


In [43]:
screen_time_churn = (
    df_first_session[df_first_session["Ушел из приложения"] == 1]
    .groupby(["Функционал"])
    .agg(
        mean_time=("mean_time_per_event", "mean"),
        median_time=("mean_time_per_event", "median"),
        n_events=("Функционал", "count")
    )
    .query("n_events >= 1000")   # фильтр от шума
    .sort_values("mean_time", ascending=False)
)

screen_time_churn.head(10)

,mean_time,median_time,n_events
Функционал,,,
Открытие описания варианта умного решения,0.484808,0.4,1119
Переход в раздел 'Опросы и собрания собственников',0.406832,0.3,11315
Переход в раздел 'Профиль',0.389509,0.3,3460
Отправка заявки,0.369459,0.3,3916
Просмотр уведомления,0.369149,0.3,25840
Переход в раздел 'Мой транспорт',0.359724,0.3,3481
Переход в раздел 'Приборы учета',0.356467,0.2,2915
Выбор актуальной темы,0.344024,0.2,1690
Открытие экрана,0.309014,0.2,65108


In [44]:
screens_of_interest = screen_time_churn.head(5).index

screen_time_compare = (
    df_first_session[df_first_session["Функционал"].isin(screens_of_interest)]
    .groupby(["Ушел из приложения", "Функционал"])
    .agg(
        mean_time=("mean_time_per_event", "mean"),
        median_time=("mean_time_per_event", "median"),
        n_events=("Функционал", "count")
    )
    .reset_index()
)

screen_time_compare

,Ушел из приложения,Функционал,mean_time,median_time,n_events
0,0,Открытие описания варианта умного решения,0.496064,0.4,813
1,0,Отправка заявки,0.366937,0.3,5541
2,0,Переход в раздел 'Опросы и собрания собственни...,0.409902,0.3,8241
3,0,Переход в раздел 'Профиль',0.431467,0.3,2625
4,0,Просмотр уведомления,0.355648,0.2,25257
5,1,Открытие описания варианта умного решения,0.484808,0.4,1119
6,1,Отправка заявки,0.369459,0.3,3916
7,1,Переход в раздел 'Опросы и собрания собственни...,0.406832,0.3,11315
8,1,Переход в раздел 'Профиль',0.389509,0.3,3460
9,1,Просмотр уведомления,0.369149,0.3,25840


In [109]:
df_sessions = (
    df_clean
    .drop_duplicates(
        subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )
    [[
        "Идентификатор устройства",
        "Номер сессии в рамках устройства",
        "events_cnt",
        'mean_time_per_event',
        "Ушел из приложения"
    ]]
)
df_churn_sessions = df_sessions[df_sessions['Ушел из приложения'] == 1]
user_short_only = (
    df_churn_sessions
    .groupby("Идентификатор устройства")
    .agg(
        max_events=('events_cnt', 'max')
    )
    .reset_index()
)
user_short_only = user_short_only[user_short_only['max_events'] <= 2]

user_mean_time = (
    df_churn_sessions
    .merge(
        user_short_only[['Идентификатор устройства']],
        on='Идентификатор устройства',
        how='inner'
    )
    .groupby("Идентификатор устройства")
    .agg(
        max_time_per_event=('mean_time_per_event', 'max')
    )
    .reset_index()
)

user_mean_time['is_instant'] = (
    user_mean_time['max_time_per_event'] <= 6 / 60
)

share_instant = user_mean_time['is_instant'].mean() * 100
share_instant

np.float64(81.19930389641034)

In [113]:
print(f'Из всех ушедших пользователей только с короткими сессиями (а это 50% от всех ушедших) {round(share_instant, 2)}% не совершали действий длиннее 6 секунд ')

Из всех ушедших пользователей только с короткими сессиями (а это 50% от всех ушедших) 81.2% не совершали действий длиннее 6 секунд 


In [ ]:
df_short.groupby(["Идентификатор устройства",
        "Номер сессии в рамках устройства",]'Ушел из приложения')['Длина сессии в минутах'].median()

In [199]:
os_time = (
    more2events
    .groupby("ОС")["time_per_event"]
    .agg(
        mean_time="mean",
        median_time="median",
        events_cnt="count"
    )
    .sort_values("median_time", ascending=False)
)

os_time

,mean_time,median_time,events_cnt
ОС,,,
iOS,0.290271,0.196667,698671
Android,0.194281,0.133333,1380013


In [209]:
leave_ios = more2events[(more2events['Ушел из приложения']==1) & (more2events['ОС']=='iOS')].shape[0]
leave_android = more2events[(more2events['Ушел из приложения']==1) & (more2events['ОС']=='Android')].shape[0]
stay_ios = more2events[(more2events['Ушел из приложения']==0) & (more2events['ОС']=='iOS')].shape[0]
stay_android = more2events[(more2events['Ушел из приложения']==0) & (more2events['ОС']=='Android')].shape[0]
print(stay_ios/leave_ios, stay_android/leave_android)

4.772375389343755 4.605889378158361


In [204]:
top_models = (
    more2events["Модель устройства"]
    .value_counts()
    .head(30)
    .index
)
model_time = (
    more2events[more2events["Модель устройства"].isin(top_models)]
    .groupby("Модель устройства")["time_per_event"]
    .agg(
        mean_time="mean",
        median_time="median",
        events_cnt="count"
    )
    .sort_values("median_time", ascending=False)
)

model_time

,mean_time,median_time,events_cnt
Модель устройства,,,
iPhone XR,0.319524,0.216667,19851
iPhone 11,0.306391,0.211111,61354
iPhone 12 Pro Max,0.295084,0.203030,19078
iPhone 12 Pro,0.295795,0.202778,16490
iPhone 12,0.296072,0.200000,29098
iPhone 14 Pro Max,0.285623,0.200000,41603
iPhone 13 Pro Max,0.288986,0.196667,36435
iPhone 13,0.287745,0.195833,49219
iPhone 15 Pro Max,0.279728,0.195000,53270


In [91]:
more2events = df_clean[df_clean['events_cnt']>2]

In [191]:
screen_pop_events = (
    more2events[more2events["Ушел из приложения"] == 1]
    ["Экран"]
    .value_counts(normalize=True)   # доля; убери normalize=True, если нужны абсолюты
    .rename("share")
    .reset_index()
    .rename(columns={"index": "Экран"})
)

screen_pop_events.head(20)


,Экран,share
0,Еще,0.449725
1,Новая заявка,0.327280
2,Важное,0.106528
3,Услуги,0.029969
4,Заявки,0.023262
5,Мои баллы,0.021323
6,Мой дом,0.013488
7,Объявления,0.011048
8,Новое ОСС,0.010275
9,Связаться с владельцем ТС,0.003600


In [192]:
screen_pop_events = (
    more2events[more2events["Ушел из приложения"] == 0]
    ["Экран"]
    .value_counts(normalize=True)   # доля; убери normalize=True, если нужны абсолюты
    .rename("share")
    .reset_index()
    .rename(columns={"index": "Экран"})
)

screen_pop_events.head(20)

,Экран,share
0,Еще,0.400564
1,Новая заявка,0.350868
2,Заявки,0.135306
3,Важное,0.052448
4,Услуги,0.014892
5,Мои баллы,0.014400
6,Мой дом,0.012829
7,Объявления,0.006900
8,Новое ОСС,0.005745
9,Гостевой доступ,0.004181


In [197]:
screen_pop_users = (
    more2events[more2events["Ушел из приложения"] == 1]
    .groupby("Экран")["Идентификатор устройства"]
    .nunique()
    .sort_values(ascending=False)
    .rename("unique_devices")
    .reset_index()
)

screen_pop_users.head(20)


,Экран,unique_devices
0,Еще,30607
1,Важное,15755
2,Новая заявка,11551
3,Мои баллы,3825
4,Услуги,3550
5,Объявления,2200
6,Новое ОСС,1368
7,Связаться с владельцем ТС,1028
8,Мой дом,997
9,Заявки,974


In [198]:
screen_pop_users = (
    more2events[more2events["Ушел из приложения"] == 0]
    .groupby("Экран")["Идентификатор устройства"]
    .nunique()
    .sort_values(ascending=False)
    .rename("unique_devices")
    .reset_index()
)

screen_pop_users.head(20)

,Экран,unique_devices
0,Еще,48966
1,Важное,27318
2,Новая заявка,24951
3,Заявки,11802
4,Мои баллы,8863
5,Услуги,7778
6,Объявления,5441
7,Мой дом,4060
8,Новое ОСС,2326
9,Связаться с владельцем ТС,1951


In [92]:
session_level = more2events.drop_duplicates(
    subset=["Идентификатор устройства", "Номер сессии в рамках устройства"]
)
age_churn_stats = (
    session_level
    .groupby(["age_group", "Ушел из приложения"])["Длина сессии в минутах"]
    .agg(
        mean_duration="mean",
        median_duration="median",
        count="count"
    )
    .reset_index()
)
age_churn_stats

,age_group,Ушел из приложения,mean_duration,median_duration,count
0,18–24,0,1.196526,0.666667,5373
1,18–24,1,1.557154,1.016667,1696
2,25–30,0,1.243063,0.750000,15545
3,25–30,1,1.554483,0.966667,3993
4,31-40,0,1.260303,0.716667,73905
5,31-40,1,1.597146,1.016667,17437
6,41–50,0,1.447981,0.816667,75276
7,41–50,1,1.877647,1.166667,17157
8,51–60,0,1.632112,0.883333,45420
9,51–60,1,2.208777,1.300000,8762


In [212]:
more2events.groupby(
    ["age_group", "Ушел из приложения"]
)["events_cnt"].median()


age_group  Ушел из приложения
18–24      0                     8.0
           1                     7.0
25–30      0                     8.0
           1                     8.0
31-40      0                     8.0
           1                     8.0
41–50      0                     8.0
           1                     8.0
51–60      0                     8.0
           1                     8.0
61–70      0                     8.0
           1                     8.0
71–101     0                     7.0
           1                     8.0
Name: events_cnt, dtype: float64

In [102]:
more2events["time_per_event"] = (
    more2events["Длина сессии в минутах"] /
    more2events["events_cnt"]
)
time_per_event_stats = (
    more2events
    .groupby(["age_group", "Ушел из приложения"])["time_per_event"]
    .agg(
        mean_time_per_event="mean",
        median_time_per_event="median",
        count="count"
    )
    .reset_index()
)

time_per_event_stats


/tmp/ipython-input-256853283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more2events["time_per_event"] = (


,age_group,Ушел из приложения,mean_time_per_event,median_time_per_event,count
0,18–24,0,0.180279,0.122222,35661
1,18–24,1,0.258030,0.178571,10235
2,25–30,0,0.180138,0.128788,107270
3,25–30,1,0.241764,0.164286,25674
4,31-40,0,0.181116,0.127778,514271
5,31-40,1,0.246081,0.176190,113172
6,41–50,0,0.207835,0.144444,524446
7,41–50,1,0.280793,0.200000,114728
8,51–60,0,0.242494,0.165385,305701
9,51–60,1,0.334722,0.227778,57819


In [107]:
more2events['Дата и время события'] = pd.to_datetime(more2events['Дата и время события'])

/tmp/ipython-input-3983115427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more2events['Дата и время события'] = pd.to_datetime(more2events['Дата и время события'])


In [108]:
m2e_sorted = more2events.sort_values([
    "Идентификатор устройства",
    "Номер сессии в рамках устройства",
    "Дата и время события"
])

In [109]:
m2e_sorted["next_event_time"] = (
    m2e_sorted
    .groupby(
        ["Идентификатор устройства", "Номер сессии в рамках устройства"]
    )["Дата и время события"]
    .shift(-1)
)

In [110]:
m2e_sorted["time_to_next_event_sec"] = (
    m2e_sorted["next_event_time"] - m2e_sorted["Дата и время события"]
).dt.total_seconds()

In [112]:
m2e_sorted = m2e_sorted[m2e_sorted["time_to_next_event_sec"].notna()]
m2e_sorted = m2e_sorted[m2e_sorted["time_to_next_event_sec"] > 0]

In [175]:
screen_func_time = (
    m2e_sorted
    .groupby(["Экран", "Функционал"])["time_to_next_event_sec"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)
top20_long_actions = screen_func_time.head(20)
top20_long_actions

,Экран,Функционал,time_to_next_event_sec
0,Гостевой доступ,Отказ гостя от предоставленного гостевого доступа,764.000000
1,Новое ОСС,Прочтение справки на шаге порядка приема админ...,178.500000
2,Объявления,Редактирование опубликованного объявления,177.076923
3,Новое ОСС,Прочтение инструкции об обязанностях администр...,140.500000
4,Новое ОСС,Прочтение справки на шаге выбора администратор...,124.000000
5,Новое ОСС,Прочтение справки на шаге времени ознакомления,115.157895
6,Новое ОСС,Добавление вопроса,78.928571
7,Заявки,Опровержение выполнения заявки,78.439730
8,Еще,Переход в раздел 'База знаний',75.193496
9,Мои баллы,Переход к поощрениям,72.968496


In [188]:
screen_func_time = (
    m2e_sorted[m2e_sorted['Ушел из приложения']==1]
    .groupby(["Экран"])
    .agg(
        mean_time_sec=("time_to_next_event_sec", "mean"),
        median_time_sec=("time_to_next_event_sec", "median"),
        events_count=("time_to_next_event_sec", "count")
    )
    .reset_index()
)
screen_func_time_filtered = (
    screen_func_time
    [screen_func_time["events_count"] >= 100]  # порог подбери
    .sort_values("events_count", ascending=False)
)

screen_func_time_filtered.head(20)

,Экран,mean_time_sec,median_time_sec,events_count
2,Еще,24.412096,7.0,119613
6,Новая заявка,17.477600,7.0,98863
0,Важное,31.254082,12.0,32580
12,Услуги,23.827849,12.0,8934
4,Мои баллы,16.036114,8.0,5538
3,Заявки,13.971863,8.0,4798
5,Мой дом,20.305934,11.0,4112
7,Новое ОСС,42.394893,20.0,2859
9,Объявления,11.685262,3.0,2653
10,Связаться с владельцем ТС,51.216216,27.0,888


In [189]:
screen_func_time = (
    m2e_sorted[m2e_sorted['Ушел из приложения']==0]
    .groupby(["Экран"])
    .agg(
        mean_time_sec=("time_to_next_event_sec", "mean"),
        median_time_sec=("time_to_next_event_sec", "median"),
        events_count=("time_to_next_event_sec", "count")
    )
    .reset_index()
)
screen_func_time_filtered = (
    screen_func_time
    [screen_func_time["events_count"] >= 100]  # порог подбери
    .sort_values("events_count", ascending=False)
)

screen_func_time_filtered.head(20)

,Экран,mean_time_sec,median_time_sec,events_count
6,Новая заявка,16.946546,7.0,476000
2,Еще,18.744909,5.0,457758
3,Заявки,12.706771,7.0,136927
0,Важное,26.521594,11.0,73818
12,Услуги,20.937693,11.0,20046
5,Мой дом,19.501355,10.0,18083
4,Мои баллы,16.336250,7.0,16916
7,Новое ОСС,37.728360,13.0,7359
9,Объявления,9.736098,3.0,7211
1,Гостевой доступ,19.387741,8.0,4405


In [178]:
compare = (
    m2e_sorted
    .groupby(["Экран", "Функционал", "Ушел из приложения"])
    ["time_to_next_event_sec"]
    .median()
    .unstack()
)

compare["diff"] = compare[1] - compare[0]
compare.sort_values("diff", ascending=False).head(20)

Ушел из приложения                                                     0  \
Экран           Функционал                                                 
Объявления      Редактирование опубликованного объявления           49.0   
Новое ОСС       Прочтение справки на шаге времени ознакомления      64.5   
                Ознакомление с обязанностями администратора пос...   2.0   
                Переход к настройке и описанию ОСС                   3.0   
Новый адрес     Раскрытие вкладки 'Ввод адреса'                     13.0   
Мой дом         Переход к редактированию адреса                     25.5   
Гостевой доступ Переход к активным гостевым доступам после заве...  11.5   
Мои баллы       Получение промокода                                  4.0   
Новый адрес     Переход в справку по активации гостевого доступа    16.0   
Услуги          Отмена подачи заявки на подключение умного решения   5.0   
Заявки          Опровержение выполнения заявки                      56.0   
Мой дом         Переход в раздел 'Мои соседи'                       34.0   
Новый адрес     Открытие экрана                                     11.0   
Мой дом         Переход к открытию шлагбаума                        15.0   
Новое ОСС       Переход к согласию о доступности персональных д...  24.0   
Новый адрес     Добавление нового адреса                             7.0   
Мои баллы       Переход на портал 'Миллион призов'                   8.0   
Гостевой доступ Переход в справку по функционалу доступов           16.0   
Мой дом         Включение/выключение отображения в списке            4.0   
Гостевой доступ Переход к предоставлению доступа через +             4.0   

Ушел из приложения                                                      1  \
Экран           Функционал                                                  
Объявления      Редактирование опубликованного объявления           385.0   
Новое ОСС       Прочтение справки на шаге времени ознакомления      116.0   
                Ознакомление с обязанностями администратора пос...   26.0   
                Переход к настройке и описанию ОСС                   23.5   
Новый адрес     Раскрытие вкладки 'Ввод адреса'                      31.5   
Мой дом         Переход к редактированию адреса                      39.0   
Гостевой доступ Переход к активным гостевым доступам после заве...   23.0   
Мои баллы       Получение промокода                                  15.5   
Новый адрес     Переход в справку по активации гостевого доступа     27.0   
Услуги          Отмена подачи заявки на подключение умного решения   15.0   
Заявки          Опровержение выполнения заявки                       66.0   
Мой дом         Переход в раздел 'Мои соседи'                        44.0   
Новый адрес     Открытие экрана                                      20.0   
Мой дом         Переход к открытию шлагбаума                         24.0   
Новое ОСС       Переход к согласию о доступности персональных д...   32.0   
Новый адрес     Добавление нового адреса                             15.0   
Мои баллы       Переход на портал 'Миллион призов'                   15.0   
Гостевой доступ Переход в справку по функционалу доступов            23.0   
Мой дом         Включение/выключение отображения в списке            11.0   
Гостевой доступ Переход к предоставлению доступа через +              9.5   

Ушел из приложения                                                   diff  
Экран           Функционал                                                 
Объявления      Редактирование опубликованного объявления           336.0  
Новое ОСС       Прочтение справки на шаге времени ознакомления       51.5  
                Ознакомление с обязанностями администратора пос...   24.0  
                Переход к настройке и описанию ОСС                   20.5  
Новый адрес     Раскрытие вкладки 'Ввод адреса'                      18.5  
Мой дом         Переход к редактированию адреса                      13.5  
Гостевой доступ Переход 

In [123]:
m2e_sorted.groupby(
    ["Ушел из приложения"]
)["time_to_next_event_sec"].mean()


,time_to_next_event_sec
Ушел из приложения,
0,17.961042
1,22.482437


In [164]:
m2e_sorted["is_last_event"] = (
    m2e_sorted.groupby("Идентификатор устройства")["Дата и время события"]
      .transform("max")
    == m2e_sorted["Дата и время события"]
)
m2e_sorted["is_penultimate_event"] = (
    m2e_sorted.groupby("Идентификатор устройства")["Дата и время события"]
    .rank(method="first", ascending=False)
    == 2
)

In [165]:
m2e_sorted_1 = m2e_sorted[m2e_sorted['Ушел из приложения']==1]
m2e_sorted_0 = m2e_sorted[m2e_sorted['Ушел из приложения']==0]

In [172]:
m2e_sorted_1[m2e_sorted_1['is_penultimate_event']==0]['time_to_next_event_sec'].median()

8.0

In [173]:
m2e_sorted_0[m2e_sorted_0['is_penultimate_event']==0]['time_to_next_event_sec'].median()

7.0

In [174]:
m2e_sorted_1[m2e_sorted_1['is_last_event']==1]

,Unnamed: 0,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,...,is_churn,is_finish,age_group,Длина сессии в минутах,events_cnt,time_per_event,next_event_time,time_to_next_event_sec,is_last_event,is_penultimate_event
2632606,2636275,2025-10-24 16:16:31+03:00,Еще,Переход в раздел 'Приборы учета',Тап на кнопку 'Приборы учёта',10,10000000374,Samsung,Galaxy A04,phone,...,0,0,41–50,0.433333,3,0.144444,2025-10-24 16:16:55+03:00,24.0,True,False
17533,17536,2025-10-20 15:00:07+03:00,Еще,Открытие экрана,Переход к экрану,29,10000000736,Tecno,TECNO CAMON 20,phone,...,0,0,61–70,3.100000,4,0.775000,2025-10-20 15:00:10+03:00,3.0,True,False
17542,17545,2025-10-20 08:40:30+03:00,Заявки,Переход к просмотру заявки,Тап на заявку,31,10000000171,Apple,iPhone 11,phone,...,0,0,25–30,1.466667,8,0.183333,2025-10-20 08:40:34+03:00,4.0,True,False
7050,7050,2025-10-25 10:14:33+03:00,Еще,Открытие экрана,Переход к экрану,62,10000000068,Samsung,Galaxy S23,phone,...,0,0,61–70,0.966667,6,0.161111,2025-10-25 10:14:39+03:00,6.0,True,False
2073737,2076734,2025-09-13 20:17:21+03:00,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников',63,10000000044,Samsung,Galaxy A23,phone,...,0,0,41–50,1.650000,5,0.330000,2025-09-13 20:18:30+03:00,69.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840624,2845015,2025-09-18 19:35:56+03:00,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников',180273,10000000070,Realme,realme 9 Pro+,phone,...,0,0,51–60,0.500000,3,0.166667,2025-09-18 19:36:23+03:00,27.0,True,False
2840823,2845214,2025-09-18 15:59:48+03:00,Еще,Открытие экрана,Переход к экрану,180284,10000000250,Doogee,V20,phone,...,0,0,51–60,0.166667,3,0.055556,2025-09-18 15:59:51+03:00,3.0,True,False
2747168,2751057,2025-09-13 15:43:13+03:00,Новая заявка,Открытие экрана,Переход к экрану,180285,10000000002,Samsung,Galaxy S22 Ultra,phone,...,0,0,31-40,0.100000,4,0.025000,2025-09-13 15:43:15+03:00,2.0,True,False
2747180,2751069,2025-09-13 21:53:28+03:00,Еще,Переход в раздел 'Мои баллы',Тап на кнопку 'Мои баллы',180286,10000000178,Realme,realme GT 2 Pro,phone,...,0,0,31-40,1.000000,8,0.125000,2025-09-13 21:53:34+03:00,6.0,True,False


In [150]:
df_sorted_desc = m2e_sorted_1.sort_values(by='time_to_next_event_sec', ascending=False)
df_sorted_desc

,Unnamed: 0,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,...,row_id,Ушел из приложения,is_churn,is_finish,age_group,Длина сессии в минутах,events_cnt,time_per_event,next_event_time,time_to_next_event_sec
1722560,1725148,2025-09-23 17:42:44+03:00,Еще,Открытие экрана,Переход к экрану,60946,10000000002,Apple,iPhone 12 Pro,phone,...,1998505,1,0,0,25–30,54.633333,5,10.926667,2025-09-23 18:35:11+03:00,3147.0
213724,213928,2025-09-03 09:10:37+03:00,Еще,Переход в раздел 'База знаний',Тап на кнопку 'База знаний',24706,10000000004,Samsung,Galaxy A8(2018),phone,...,245793,1,0,0,41–50,56.600000,8,7.075000,2025-09-03 09:58:35+03:00,2878.0
848071,849355,2025-09-30 18:06:20+03:00,Объявления,Открытие экрана,Открытие экрана,111395,10000000013,Google,Pixel 5,phone,...,983615,1,0,0,51–60,46.033333,6,7.672222,2025-09-30 18:52:15+03:00,2755.0
536994,537714,2025-09-22 15:56:47+03:00,Услуги,Открытие экрана,Открытие экрана,71208,10000000136,Google,Chromebook Pixel (2015),phone,...,616751,1,0,0,NaN,45.316667,7,6.473810,2025-09-22 16:38:03+03:00,2476.0
1722557,1725145,2025-09-23 17:04:27+03:00,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников',60946,10000000001,Apple,iPhone 12 Pro,phone,...,1998502,1,0,0,25–30,39.516667,4,9.879167,2025-09-23 17:43:37+03:00,2350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999937,2002857,2025-09-06 21:50:08+03:00,Новая заявка,Вызов поиска,Тап на кнопку поиска,159219,10000000023,Apple,iPhone 13 Pro Max,phone,...,2330965,1,0,0,41–50,4.233333,23,0.184058,2025-09-06 21:50:09+03:00,1.0
1999940,2002860,2025-09-06 21:50:28+03:00,Новая заявка,Вызов поиска,Тап на кнопку поиска,159219,10000000023,Apple,iPhone 13 Pro Max,phone,...,2330968,1,0,0,41–50,4.233333,23,0.184058,2025-09-06 21:50:29+03:00,1.0
1999942,2002862,2025-09-06 21:50:31+03:00,Новая заявка,Вызов поиска,Тап на кнопку поиска,159219,10000000023,Apple,iPhone 13 Pro Max,phone,...,2330970,1,0,0,41–50,4.233333,23,0.184058,2025-09-06 21:50:32+03:00,1.0
1999943,2002863,2025-09-06 21:50:32+03:00,Новая заявка,Вызов поиска,Тап на кнопку поиска,159219,10000000023,Apple,iPhone 13 Pro Max,phone,...,2330971,1,0,0,41–50,4.233333,23,0.184058,2025-09-06 21:50:33+03:00,1.0
